![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_ALBERT.ipynb)

# Import ONNX ALBERT models from HuggingFace 🤗 into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in  `Spark NLP 5.0.0`, enabling high performance inference for models. Please make sure you have upgraded to the latest Spark NLP release.
- You can import models for ALBERT from HuggingFace and they have to be in `Fill Mask` category. Meaning, you cannot use ALBERT models trained/fine-tuned on a specific task such as token/sequence classification.

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.48.2`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [ ]:
!pip install -q --upgrade transformers[onnx]==4.48.2 optimum onnx

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- We'll use [albert-base-v2](https://huggingface.co/albert-base-v2 ) model from HuggingFace as an example and load it as a `ORTModelForFeatureExtraction`, representing an ONNX model.
- In addition to the ALBERT model, we also need to save the `AlbertTokenizer`. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [2]:
from transformers import AutoTokenizer
from optimum.onnxruntime import ORTModelForFeatureExtraction

MODEL_NAME = "albert-base-v2"
EXPORT_PATH = f"onnx_models/{MODEL_NAME}"

ort_model = ORTModelForFeatureExtraction.from_pretrained(MODEL_NAME, export=True)
ort_model.save_pretrained(EXPORT_PATH)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained(EXPORT_PATH)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

('onnx_models/albert-base-v2/tokenizer_config.json',
 'onnx_models/albert-base-v2/special_tokens_map.json',
 'onnx_models/albert-base-v2/spiece.model',
 'onnx_models/albert-base-v2/added_tokens.json',
 'onnx_models/albert-base-v2/tokenizer.json')

Let's have a look inside these two directories and see what we are dealing with:

In [3]:
!ls -l {EXPORT_PATH}

total 46704
-rw-r--r-- 1 root root      798 Jun  7 05:26 config.json
-rw-r--r-- 1 root root 44777285 Jun  7 05:26 model.onnx
-rw-r--r-- 1 root root      286 Jun  7 05:26 special_tokens_map.json
-rw-r--r-- 1 root root   760289 Jun  7 05:26 spiece.model
-rw-r--r-- 1 root root     1252 Jun  7 05:26 tokenizer_config.json
-rw-r--r-- 1 root root  2272346 Jun  7 05:26 tokenizer.json


- We need to move the `spiece.model` file from the tokenizer into an assets folder, as this is where Spark NLP looks for it when working with models like Albert or other SentencePiece-based tokenizers.

In [4]:
!mkdir -p {EXPORT_PATH}/assets && mv {EXPORT_PATH}/spiece.model {EXPORT_PATH}/assets/

In [5]:
!ls -l {EXPORT_PATH}/assets

total 744
-rw-r--r-- 1 root root 760289 Jun  7 05:26 spiece.model


## Import and Save ALBERT in Spark NLP

Let's install and setup Spark NLP in Google Colab. For this example, we'll use specific versions of `pyspark` and `spark-nlp` that we've already tested with this transformer model to make sure everything runs smoothly:

If you prefer to use the latest versions, feel free to run:

`!pip install -q pyspark spark-nlp`

Just keep in mind that newer versions might have some changes, so you may need to tweak your code a bit if anything breaks.

In [6]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 31.5 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [7]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  5.5.3
Apache Spark version:  3.5.4


- Let's use `loadSavedModel` functon in `AlbertEmbeddings` which allows us to load the ONNX model
- Most params will be set automatically. They can also be set later after loading the model in `AlbertEmbeddings` during runtime, so don't worry about setting them now
- `loadSavedModel` accepts two params, first is the path to the exported model. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- `setStorageRef` is very important. When you are training a task like NER or any Text Classification, we use this reference to bound the trained model to this specific embeddings so you won't load a different embeddings by mistake and see terrible results 😊
- It's up to you what you put in `setStorageRef` but it cannot be changed later on. We usually use the name of the model to be clear, but you can get creative if you want!
- The `dimension` param is is purely cosmetic and won't change anything. It's mostly for you to know later via `.getDimension` what is the dimension of your model. So set this accordingly.
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.


In [8]:
from sparknlp.annotator import AlbertEmbeddings

albert = AlbertEmbeddings.loadSavedModel(f"{EXPORT_PATH}", spark)\
    .setInputCols(["sentence",'token'])\
    .setOutputCol("albert")\
    .setCaseSensitive(True)\
    .setDimension(768)\
    .setStorageRef('albert_base_uncased')

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [9]:
albert.write().overwrite().save(f"{MODEL_NAME}_spark_nlp")

Let's clean up stuff we don't need anymore

In [10]:
!rm -rf {EXPORT_PATH}

Awesome  😎 !

This is your ONNX ALBERT model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [11]:
! ls -l {MODEL_NAME}_spark_nlp

total 44484
-rw-r--r-- 1 root root 44784254 Jun  7 05:32 albert_onnx
-rw-r--r-- 1 root root   760289 Jun  7 05:32 albert_spp
drwxr-xr-x 2 root root     4096 Jun  7 05:32 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny ALBERT model 😊

In [12]:
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import Tokenizer, AlbertEmbeddings
from pyspark.ml import Pipeline

document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols("document") \
    .setOutputCol("token")

albert = AlbertEmbeddings.load(f"{MODEL_NAME}_spark_nlp") \
    .setInputCols(["document", "token"]) \
    .setOutputCol("albert")

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    albert
])

data = spark.createDataFrame([["William Henry Gates III (born October 28, 1955) is an American business magnate, software developer, investor, and philanthropist."]], ["text"])

model = pipeline.fit(data)
result = model.transform(data)

result.selectExpr("explode(albert.embeddings) as embeddings").show()

+--------------------+
|          embeddings|
+--------------------+
|[-0.6758107, -0.3...|
|[-0.8178196, -1.7...|
|[-0.8304708, -1.7...|
|[-0.6008072, -1.0...|
|[0.059836745, -1....|
|[-0.70512545, -0....|
|[-0.5015387, -1.0...|
|[1.5703614, -0.66...|
|[0.41954327, -1.0...|
|[0.17391497, -1.7...|
|[0.92069304, -0.3...|
|[1.0872071, -0.07...|
|[1.0310556, 0.092...|
|[-1.115605, -1.20...|
|[-1.4810971, 0.36...|
|[-1.194795, -0.21...|
|[1.3548697, -1.10...|
|[-0.5084445, 0.03...|
|[-1.2231891, -0.7...|
|[0.6835313, -1.15...|
+--------------------+
only showing top 20 rows



That's it! You can now go wild and use hundreds of ALBERT models from HuggingFace 🤗 in Spark NLP 🚀
